In [1]:
val raw = spark.read.parquet("file:///home/whsh/part-r-00000-c96a5543-d316-4567-8cdb-dec24abe1f85.gz.parquet")
raw.printSchema

root
 |-- UT: string (nullable = true)
 |-- FUID: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- institutions: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- query: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- q: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |    |    |-- count: integer (nullable = true)
 |-- json: string (nullable = true)



In [11]:
raw.select("query").take(2).foreach(println)

[WrappedArray([Istanbul Univ,AD,1], [ULUBELEN A*,AU,1], [Phytochemistry,SO,1], [Ulubelen A,AU,2], [TUBITAK,AD,1], [Longipinene derivatives from Lavandula stoechas subsp. stoechas,TI,1], [ULUBELEN,A*,AU,1])]
[WrappedArray([Enderlein, J,AU,1], [myosin,TI,1], [Defocused image,TI,1], [Myosin,TI,2], [Syed,AU,2], [selvin,AU,1], [mckinney AND ha,AU,1], [selvin AND ha,AU,2], [Enderlein AND Selvin,AU,2], [quantum dots image,TI,2], [selvin pr AND ha t,AU,2], [quantum dots  defocused imaging,TS,1], [Selvin P,AU,1], [goldman,AU,1], [focus,TI,1])]


In [10]:
val newdf = raw.withColumn("queryone", org.apache.spark.sql.functions.explode(raw("query")))
newdf.show

+-------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 UT|         FUID|          categories|        institutions|             authors|               query|                json|            queryone|
+-------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|WOS:A1988R564700055|11389186WOS11|["BIOCHEMISTRY MO...|["BASIC SCI RES I...|["OLCAY Y","GOREN...|[[Istanbul Univ,A...|{"publishers":[{"...|[Istanbul Univ,AD,1]|
|WOS:A1988R564700055|11389186WOS11|["BIOCHEMISTRY MO...|["BASIC SCI RES I...|["OLCAY Y","GOREN...|[[Istanbul Univ,A...|{"publishers":[{"...|  [ULUBELEN A*,AU,1]|
|WOS:A1988R564700055|11389186WOS11|["BIOCHEMISTRY MO...|["BASIC SCI RES I...|["OLCAY Y","GOREN...|[[Istanbul Univ,A...|{"publishers":[{"...|[Phytochemistry,S...|
|WOS:A1988R564700055|1138918

In [11]:
val filterdf = newdf.filter("queryone.count > 9")selectExpr("queryone.q as keyword","queryone.count as count","UT as id","json")
filterdf.show

+--------------------+-----+-------------------+--------------------+
|             keyword|count|                 id|                json|
+--------------------+-----+-------------------+--------------------+
|              nature|   16|WOS:000319254000053|{"pmid":["MEDLINE...|
|            planaria|   14|WOS:000337523700146|{"publishers":[{"...|
|planaria regenera...|   15|WOS:000337523700146|{"publishers":[{"...|
|                J. L|   21|WOS:000181900300002|{"abstract":["The...|
|                 j l|   19|WOS:000181900300002|{"abstract":["The...|
|                 J L|   75|WOS:000181900300002|{"abstract":["The...|
|Functional connec...|   12|WOS:000263863000039|{"pmid":["MEDLINE...|
|                MnBi|   16|WOS:000342245700019|{"abstract":["Fer...|
|      photocatalytic|   10|WOS:000334855500039|{"abstract":["Ag2...|
|                Ag2O|   12|WOS:000334855500039|{"abstract":["Ag2...|
|            Mitchell|   10|WOS:000262040100004|{"abstract":["The...|
|            shapoto

In [12]:
filterdf.printSchema

root
 |-- keyword: string (nullable = true)
 |-- count: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- json: string (nullable = true)



In [24]:
import org.json4s._
import org.json4s.JsonDSL._
import org.json4s.jackson.JsonMethods._

val res = filterdf.map(row=>{
    val keyword = row.getString(0)
    val count = row.getInt(1)
    val id = row.getString(2)
    val json = (parse(row.getString(3))\"title")
    val title = json(0)
    "keyword: "+keyword+",count: "+count+", id: "+id+", title: "+title.values
})
res.take(15).foreach(println)

keyword: nature,count: 16, id: WOS:000319254000053, title: Reconfiguration of the proteasome during chaperone-mediated assembly
keyword: planaria,count: 14, id: WOS:000337523700146, title: COMBINATORIAL CO-MORBIDITY IN PLANARIA EXPOSED TO CADMIUM AND ETHANOL DURING HEAD REGENERATION
keyword: planaria regeneration,count: 15, id: WOS:000337523700146, title: COMBINATORIAL CO-MORBIDITY IN PLANARIA EXPOSED TO CADMIUM AND ETHANOL DURING HEAD REGENERATION
keyword: J. L,count: 21, id: WOS:000181900300002, title: Sustained reduction of serum cholesterol in low-dose 6-year simvastatin treatment with minimum side effects in 51,321 Japanese hypercholesterolemic patients - Implication of the J-LIT study, a large scale nationwide cohort study
keyword: j l,count: 19, id: WOS:000181900300002, title: Sustained reduction of serum cholesterol in low-dose 6-year simvastatin treatment with minimum side effects in 51,321 Japanese hypercholesterolemic patients - Implication of the J-LIT study, a large scale 

In [13]:
import net.minidev.json.{JSONObject}
import net.minidev.json.parser.JSONParser

val res = filterdf.map(row=>{
    val keyword = row.getString("key")
    val count = row.getInt("count")
    val id = row.getString("id")
    val jsonParser =new JSONParser()
    val jsonObj: JSONObject = jsonParser.parse(row.getString("json")).asInstanceOf[JSONObject]
    val title = jsonObj.get("title").toString
    "keyword: "+keyword+",count: "+count+", id: "+id+", title: "+title
})
res.show

Name: Unknown Error
Message: <console>:25: error: object minidev is not a member of package net
       import net.minidev.json.{JSONObject}
                  ^
StackTrace: 